# WH300 Project 人工智慧 自動勾選 over 300 天的領用催還

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil  # shell utilities for removing older output directory
import peforth

### 定義 excel 表頭

In [2]:
# 一次定義好，避免以後分好幾次出現，到時候容易出錯、不一致、改漏掉。

          #  title       flag  type
          #  =========== ====  ==========
columns = [('Customer'     ,1, str       ),  #  0
           ('ProjectName'  ,1, str       ),  #  1
           ('PartNo'       ,1, str       ),  #  2
           ('PartName'     ,0, str       ),  #  3
           ('Barcode'      ,1, str       ),  #  4
           ('ActionPlan'   ,0, str       ),  #  5
           ('TargetDate'   ,0, str       ),  #  6
           ('PM_CFM'       ,0, str       ),  #  7
           ('MStatus'      ,0, str       ),  #  8
           ('RefNo'        ,0, str       ),  #  9
           ('StoreInClass' ,0, str       ),  # 10
           ('Borrower'     ,0, str       ),  # 11
           ('BorrowerID'   ,0, str       ),  # 12
           ('BorrowerDEPT' ,1, str       ),  # 13
           ('Dept'         ,0, str       ),  # 14
           ('QTY'          ,1, np.int32  ),  # 15
           ('Price'        ,0, np.float32),  # 16
           ('Days'         ,1, np.int32  ),  # 17
           ('Tag'          ,1, str       )]  # 18
names    = [i[0] for i in columns]          # excel 總表 worksheet column title
features = [i[0] for i in columns if i[1]]  # selected features from
dtypes = {} 
for i in columns: dtypes = {**dtypes,**{i[0]:i[2]}} 


- pd.read_excel() needs `pip install xlrd`. 
- Microsoft Excel is not required on the computer.
- 從原來 excel 表只抽出「總表」worksheet
- 「總表」worksheet 任何雜質都要去掉，都會影響 pd.read_excel()
- Tag 欄有打 1 以外的都是空白，這時要給定 na_values=None 這個 input argument <--- 用不著
- 用最簡單的 `pd.read_excel(r'c:\201712.xlsx')` 先讀讀看，成功了再把 `header=0, names=names, dtype=dtypes, na_values=None` 等這些多帶多錯的 args 加上去。
- Tag column 只標 1 其他都留白，這樣應該屬 dtype:str 而非 np.int32。
- Barcode 要 int64 才夠， int32 變成負的。


## Get training data and label

In [3]:
df = pd.read_excel('201712.xlsx', 
                   sheetname="Detail",  # 大小寫無關
                   header=0,  # Warehouse 的表多少會變動，不用它的，由本程式自己指定 names。
                   names=names, dtype=dtypes)
df = df.dropna(how="any", axis=0)  # 庫房來的資料「總表」中有「未領用」的，則「部門代碼」就會是空白的都剔掉，否則會造成類似 "Unable to get element as bytes." 的錯誤。
df[:4]

,Customer,ProjectName,PartNo,PartName,Barcode,ActionPlan,TargetDate,PM_CFM,MStatus,RefNo,StoreInClass,Borrower,BorrowerID,BorrowerDEPT,Dept,QTY,Price,Days,Tag
0,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900423,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9893.99918,1305,1
1,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900430,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9893.99918,1305,1
2,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900447,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9893.99918,1305,1
3,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900454,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9893.99918,1305,1


In [4]:
# 挑出要用到的 columns
training_data = df[features]  # 1
training_data[:4]


,Customer,ProjectName,PartNo,Barcode,BorrowerDEPT,QTY,Days,Tag
0,Molly,QP200,81.18R55.108G,2014050900423,20QK10,1,1305,1
1,Molly,QP200,81.18R55.108G,2014050900430,20QK10,1,1305,1
2,Molly,QP200,81.18R55.108G,2014050900447,20QK10,1,1305,1
3,Molly,QP200,81.18R55.108G,2014050900454,20QK10,1,1305,1


### 我看人家 preprocess 處理問題資料源的一些技巧
    
    # ------------ automobile -----------------------------
    
    # http://localhost:8888/notebooks/Documents/GitHub/Estimator-demo-using-Automobile-dataset/automobile.ipynb
    # Fill missing values in continuous columns with zeros instead of NaN.
    float_columns = [k for k,v in dtypes.items() if v == np.float32]
    df[float_columns] = df[float_columns].fillna(value=0., axis='columns')
    integer_columns = [k for k,v in dtypes.items() if v == np.intt32]
    df[float_columns] = df[integer_columns].fillna(value=0, axis='columns')
    
    # Fill missing values in continuous columns with '' instead of NaN (NaN mixed with strings is very bad for us).
    string_columns = [k for k,v in dtypes.items() if v == str]
    df[string_columns] = df[string_columns].fillna(value='', axis='columns')
    

    # ------------------------- US Census ---------------------------------
    # http://localhost:8888/notebooks/Documents/GitHub/tensorflow-workshop/examples/07_structured_data.ipynb
    # Separate the label we want to predict into its own object 
    # At the same time, we'll convert it into true/false to fix the formatting error
    census_train_label = census_train.pop('income').apply(lambda x: ">50K" in x)
    
    # Drop any rows that have missing elements
    # Of course there are other ways to handle missing data, but we'll
    # take the simplest approach here.
    census_train = census_train.dropna(how="any", axis=0)
    census_test = census_test.dropna(how="any", axis=0)


In [5]:
# 把 Tag 欄裡的 nan 與 '1' 改成 Boolean 
training_label = training_data.pop('Tag').apply(lambda x: "1" in x)
training_label[:4]

0    True
1    True
2    True
3    True
Name: Tag, dtype: bool

In [6]:
training_data[:4]

,Customer,ProjectName,PartNo,Barcode,BorrowerDEPT,QTY,Days
0,Molly,QP200,81.18R55.108G,2014050900423,20QK10,1,1305
1,Molly,QP200,81.18R55.108G,2014050900430,20QK10,1,1305
2,Molly,QP200,81.18R55.108G,2014050900447,20QK10,1,1305
3,Molly,QP200,81.18R55.108G,2014050900454,20QK10,1,1305


## Get test data and label

In [7]:
df = pd.read_excel(r'201801.xlsx', 
                   sheetname="Detail",  # 大小寫無關
                   header=0,  # Warehouse 的表多少會變動，不用它的，由本程式自己指定 names。
                   names=names, dtype=dtypes)
df = df.dropna(how="any", axis=0)
df[:4]

,Customer,ProjectName,PartNo,PartName,Barcode,ActionPlan,TargetDate,PM_CFM,MStatus,RefNo,StoreInClass,Borrower,BorrowerID,BorrowerDEPT,Dept,QTY,Price,Days,Tag
0,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900423,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,nan,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,10357.2548,1335,1
1,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900430,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,nan,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,10357.2548,1335,1
2,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900447,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,nan,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,10357.2548,1335,1
3,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900454,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,nan,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,10357.2548,1335,1


In [8]:
# 挑出要用到的 columns
test_data = df[features]  # 2
test_data[:20]


,Customer,ProjectName,PartNo,Barcode,BorrowerDEPT,QTY,Days,Tag
0,Molly,QP200,81.18R55.108G,2014050900423,20QK10,1,1335,1
1,Molly,QP200,81.18R55.108G,2014050900430,20QK10,1,1335,1
2,Molly,QP200,81.18R55.108G,2014050900447,20QK10,1,1335,1
3,Molly,QP200,81.18R55.108G,2014050900454,20QK10,1,1335,1
4,Molly,QP200,81.18R55.108G,2014050900461,20QK10,1,1335,1
5,Molly,QP200,81.18R55.108G,2014050900478,20QK10,1,1335,1
6,Molly,QP200,81.18R55.108G,2014050900485,20QK10,1,1335,1
7,Molly,QP200,81.18R55.108G,2014050900492,20QK10,1,1335,1
8,Molly,QP200,81.18R55.108G,2014050900508,20QK10,1,1335,1
9,Molly,QP200,81.18R55.108G,2014050900515,20QK10,1,1335,1


In [9]:
# 把 Tag 欄裡的 nan 與 '1' 改成 Boolean 
test_label = test_data.pop('Tag').apply(lambda x: "1" in x)
test_label[:4]

0    True
1    True
2    True
3    True
Name: Tag, dtype: bool

In [10]:
# 核對一下 shapes 
%f __main__ :> training_data.shape . cr
%f __main__ :> training_label.shape . cr
%f __main__ :> test_data.shape . cr
%f __main__ :> test_label.shape . cr    

(7375, 7)
(7375,)
(6025, 7)
(6025,)


### 解不了 expecting byte float found 的問題，用以前成功過的 wh300.csv 試試看。。。結果成功！ _debug_ 

    [x] 可能是 np.int32 tf.int32 前後不一致的關係！在此全部改用 tf.int64
        原因：AttributeError: type object 'numpy.int64' has no attribute 'is_integer'
        --> 結果 pandas 又不認得 tf.int64 --> 只好改用 str 了。 --> 還是失敗，類似症狀：Unable to get element as bytes.
        --> 證明是 data 本身有問題
    [x] 把 data 裁減到只剩 4 筆 --> 果然好了！ shit shit shit 
    [x] 把 data frame 倒成 csv 的方法，用來檢查資料
        df.to_csv(file_name, sep='\t', encoding='utf-8')
    [x] 從 https://pair-code.github.io/facets/ 看出，很多部門是空的！在庫房裡未領用的應該剔除。    
    [x] 以上問題用 df = df.dropna(how="any", axis=0) 解決了

## Prepare input function

In [11]:
# Make input function for training: 
#   num_epochs=None -> will cycle through input data forever
#   shuffle=True -> randomize order of input data
training_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=training_data, 
    y=training_label, 
    batch_size=700,  # 拿掉 batch size accuracy 可由 0.93 提高到 0.96! 執行該 function 可看到 default 就是 128 
    shuffle=True, 
    num_epochs=None,  # Repeat forever
    )
# 根據 Effective TensorFlow for Non-Experts (Google IO '17) 老師的例子， automobile dataset 的性質與 wh300 以及 iris 都差不多，
# 因此 batch size 以及 num_epochs 都照抄。

# Make input function for evaluation:
#   shuffle=False -> do not randomize input data
test_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=test_data, y=test_label, 
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


### Customer.txt (客戶) 跟 BorrowerDEPT.txt (部門代碼) 列表
    [ ] 每個月都可能有變化，使用前要先更新。

In [12]:
%%f 讀進 Customer.txt 跟 BorrowerDEPT.txt 轉成 list 
char Customer.txt readTextFile trim :> split('\n') value Customers // ( -- list ) ['Annie', 'Astro', 'BENZ', ... ]
char BorrowerDEPT.txt readTextFile trim :> split('\n') value BorrowerDEPT // ( -- list ) ['13P2K0', '13P3K0', '1CK0K0', ... ]

In [13]:
Customer = tf.feature_column.categorical_column_with_vocabulary_list(
           'Customer', peforth.execute('Customers').pop())
BorrowerDEPT = tf.feature_column.categorical_column_with_vocabulary_list(
           'BorrowerDEPT', peforth.execute('BorrowerDEPT').pop())

# 我看 project name 不該納入，大小寫不一，project name 對應好幾個料號，
# [ ] 2017-7 的資料有 107 種 project name, 設定 200 個 bucket.
ProjectName = tf.feature_column.categorical_column_with_hash_bucket(
              'ProjectName', 200)

# PartNo 比 project name 多很多， 2018-7 有 561 種, 設定 700 個 bucket.
PartNo = tf.feature_column.categorical_column_with_hash_bucket('PartNo', 700)

# 沒看到 datetime type 不知道 date 該如何設定。比較法，與其用 string 不如用 integer --> integer 用不成
# Barcode = tf.feature_column.numeric_column('Barcode',dtype=tf.int64)
Barcode = tf.feature_column.categorical_column_with_hash_bucket('Barcode', 2000) # 最老的 aging 有 1400 天，取 2000.

# QTY = tf.feature_column.numeric_column('QTY',dtype=tf.int32)
# Days = tf.feature_column.numeric_column('Days',dtype=tf.int32)
QTY = tf.feature_column.numeric_column('QTY')
Days = tf.feature_column.numeric_column('Days')


In [14]:
# [x] 可能這裡的順序不能出錯！ 不，沒關係。
feature_columns = [Customer, ProjectName, PartNo, Barcode, BorrowerDEPT, QTY, Days]
features

['Customer',
 'ProjectName',
 'PartNo',
 'Barcode',
 'BorrowerDEPT',
 'QTY',
 'Days',
 'Tag']

In [15]:
%f __main__ :> feature_columns value fc // ( -- list ) feature_columns 
%f fc count [for] t@ 1- fc :> [pop()] . cr cr [next]

_NumericColumn(key='Days', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_NumericColumn(key='QTY', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

_VocabularyListCategoricalColumn(key='BorrowerDEPT', vocabulary_list=('13P2K0', '13P3K0', '1CK0K0', '1CK210', '1CK220', '1CK310', '1CK320', '1CTK00', '1R3K00', '1R3K10', '1R3K20', '1RRK10', '1RRK20', '1S1K00', '1S1K01', '1S1K02', '1S1K03', '1S1K05', '1STK10', '1STK20', '1STL10', '1STL20', '1WK000', '20QK10', '20QK20', '2302K0', '2M0K10', '2M0K20', '2M0K30', '2S0K10', '2S0K20', '2S0K30', 'MBM1K0', 'MBRK10'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

_HashedCategoricalColumn(key='Barcode', hash_bucket_size=2000, dtype=tf.string)

_HashedCategoricalColumn(key='PartNo', hash_bucket_size=700, dtype=tf.string)

_HashedCategoricalColumn(key='ProjectName', hash_bucket_size=200, dtype=tf.string)

_VocabularyListCategoricalColumn(key='Customer', vocabulary_list=('Annie', 'Astro', 'BENZ', 'Ho

# 跑一種 classifier 然後跳下去訓練

# LinearClassifier

In [16]:
shutil.rmtree("model/linear", ignore_errors=True)
classifier = tf.estimator.LinearClassifier(
    feature_columns = feature_columns, 
    n_classes = 2,
    model_dir="model/linear")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/linear', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F44979F908>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# DNNClassifier

#### DNNClassifier 要求 "Items of feature_columns must be a _DenseColumn."

    c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\feature_column\feature_column.py in input_layer(features, feature_columns, weight_collections, trainable)
        211           'Items of feature_columns must be a _DenseColumn. '
        212           'You can wrap a categorical column with an '
    --> 213           'embedding_column or indicator_column. Given: {}'.format(column))
        214   weight_collections = list(weight_collections or [])
        215   if ops.GraphKeys.GLOBAL_VARIABLES not in weight_collections:

    ValueError: Items of feature_columns must be a _DenseColumn. You can wrap a categorical column with an embedding_column or indicator_column. Given: _VocabularyListCategoricalColumn(key='Customer', vocabulary_list=('Annie', 'Astro', 'Lily', 'Molly', 'others', 'Rosa', 'Venice'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

上面已經講了，所謂 DenseColumn 就是把 categorical column 都包成 embedding_column or indicator_column. 所以前面的基礎上要再加工：

# 用選定的 classifier 開始訓練

In [17]:
%pdb off
classifier.train(input_fn=training_input_fn, steps=2000)

Automatic pdb calling has been turned OFF
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into model/linear\model.ckpt.
INFO:tensorflow:loss = 485.202, step = 1
INFO:tensorflow:global_step/sec: 71.634
INFO:tensorflow:loss = 1501.49, step = 101 (1.399 sec)
INFO:tensorflow:global_step/sec: 84.4754
INFO:tensorflow:loss = 13.9387, step = 201 (1.184 sec)
INFO:tensorflow:global_step/sec: 85.4028
INFO:tensorflow:loss = 3.01202, step = 301 (1.170 sec)
INFO:tensorflow:global_step/sec: 102.592
INFO:tensorflow:loss = 6.65974, step = 401 (0.975 sec)
INFO:tensorflow:global_step/sec: 117.175
INFO:tensorflow:loss = 2.1756, step = 501 (0.853 sec)
INFO:tensorflow:global_step/sec: 116.986
INFO:tensorflow:loss = 4.05372, step = 601 (0.854 sec)
INFO:tensorflow:global_step/sec: 104.622
INFO:tensorflow:loss = 1.44792, step = 701 (0.958 sec)
INFO:tensorflow:global_step/sec: 106.11
INFO:tensorflow:loss = 3.22874, step = 801 (0.942 sec)
INFO:tensorflow:global_step/sec: 114.

In [18]:
# [x] _debug_
# 交叉試驗，看以下 evaluate 用 test_input_fn 
# 為何說是 INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InternalError'>, Unable to get element as bytes.
# 實驗結果，只要是 classifier.evaluate() 就會出這錯，兩個 input_fn 都一樣。不知何故了！
# training 跟 test 互換好像應該互換 data 檔，而非直接互換。因為 training 跟 test 的特性不一樣。
# [x] 可能是 vocabulary 有錯？
# [x] 最後發現是總表有很多未領用的物料，其「部門」為空白 NA 造成這個問題。
classifier.evaluate(input_fn=test_input_fn)


INFO:tensorflow:Starting evaluation at 2018-03-26-10:37:59
INFO:tensorflow:Restoring parameters from model/linear\model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2018-03-26-10:38:01
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.993859, accuracy_baseline = 0.975436, auc = 0.974991, auc_precision_recall = 0.878732, average_loss = 0.0261605, global_step = 2000, label/mean = 0.0245643, loss = 3.28368, prediction/mean = 0.0259237


{'accuracy': 0.99385893,
 'accuracy_baseline': 0.97543567,
 'auc': 0.9749912,
 'auc_precision_recall': 0.87873167,
 'average_loss': 0.026160471,
 'global_step': 2000,
 'label/mean': 0.024564315,
 'loss': 3.283684,
 'prediction/mean': 0.025923712}

In [19]:
# Predict multiple records
predictions = classifier.predict(test_input_fn)
# print([i['class_ids'][0] for i in predictions])
# [i for i in predictions[:10]]


In [20]:
# preds = [1 if (i['probabilities'][1] > i['probabilities'][0]) else 0 for i in predictions];
preds = [int(i['probabilities'][1] > i['probabilities'][0]) for i in predictions];
test_y = [int(i) for i in test_label]

INFO:tensorflow:Restoring parameters from model/linear\model.ckpt-2000


In [21]:
print(preds[:])    

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [22]:
# 察看一下外觀
print(np.array(preds).shape)
print(np.array(test_y).shape)

(6025,)
(6025,)


In [23]:
# 粗略比較一下 tag 的個數
print(np.sum(preds))

print(np.sum(test_y))

141
148


In [24]:
%%f  整個都印出來，用 Notepad++ 的 compare 察看，確實差不多，很成功！！！ 
__main__ :> preds tib.
__main__ :> test_y tib.    

__main__ :> preds tib. \ ==> [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### 用新月份的資料來 predict 看看

In [25]:
df = pd.read_excel('11月份W10W庫庫存報表.xlsx', 
                   sheetname="Detail",  # 大小寫無關
                   header=0,  # Warehouse 的表多少會變動，不用它的，由本程式自己指定 names。
                   names=names, dtype=dtypes)
df = df.dropna(how="any", axis=0)  # 庫房來的資料「總表」中有「未領用」的，則「部門代碼」就會是空白的都剔掉，否則會造成類似 "Unable to get element as bytes." 的錯誤。
df[:4]

,Customer,ProjectName,PartNo,PartName,Barcode,ActionPlan,TargetDate,PM_CFM,MStatus,RefNo,StoreInClass,Borrower,BorrowerID,BorrowerDEPT,Dept,QTY,Price,Days,Tag
0,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900423,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9939.258732,1272,1
1,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900430,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9939.258732,1272,1
2,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900447,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9939.258732,1272,1
3,Molly,QP200,81.18R55.108G,QP200 FOR WLAN FOR DV W/ME EC SAMPLE,2014050900454,N14:保留測試使用,2018-01-31,Keep to 2018/01/E For test,借出,F232,內部移轉入,徐濤ANDY XU#5675,K0512465,20QK10,HH QT,1,9939.258732,1272,1


In [26]:
# 挑出要用到的 columns
test_data = df[features]  # 2
test_data[:20]


,Customer,ProjectName,PartNo,Barcode,BorrowerDEPT,QTY,Days,Tag
0,Molly,QP200,81.18R55.108G,2014050900423,20QK10,1,1272,1
1,Molly,QP200,81.18R55.108G,2014050900430,20QK10,1,1272,1
2,Molly,QP200,81.18R55.108G,2014050900447,20QK10,1,1272,1
3,Molly,QP200,81.18R55.108G,2014050900454,20QK10,1,1272,1
4,Molly,QP200,81.18R55.108G,2014050900461,20QK10,1,1272,1
5,Molly,QP200,81.18R55.108G,2014050900478,20QK10,1,1272,1
6,Molly,QP200,81.18R55.108G,2014050900485,20QK10,1,1272,1
7,Molly,QP200,81.18R55.108G,2014050900492,20QK10,1,1272,1
8,Molly,QP200,81.18R55.108G,2014050900508,20QK10,1,1272,1
9,Molly,QP200,81.18R55.108G,2014050900515,20QK10,1,1272,1


In [27]:
# 把 Tag 欄裡的 nan 與 '1' 改成 Boolean 
test_label = test_data.pop('Tag').apply(lambda x: "1" in x)
test_label[:4]

0    True
1    True
2    True
3    True
Name: Tag, dtype: bool

In [28]:
# 核對一下 shapes 
%f __main__ :> test_data.shape . cr
%f __main__ :> test_label.shape . cr    

(8217, 7)
(8217,)


In [29]:
# Make input function for evaluation:
#   shuffle=False -> do not randomize input data
test_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=test_data, y=test_label, 
    num_epochs=1,  # Just one epoch
    shuffle=False, # Don't shuffle so we can compare to true labels later
    )  # 參考 GitHub/tensorflow-workshop/examples/07_structured_data.ipynb


In [30]:
# Predict 整批 test input 
preds = classifier.predict(test_input_fn)

In [31]:
# 把 predictions 由 0 與 1 的「機率」轉成 0 與 1 
predictions = [int(i['probabilities'][1] > i['probabilities'][0]) for i in preds];

INFO:tensorflow:Restoring parameters from model/linear\model.ckpt-2000


In [32]:
# 看一看預測出來的 tags 
print(predictions[:100])    

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [33]:
# 比較 tag 的個數
print(np.sum(predictions))
test_y = [int(i) for i in test_label]  # test_label 裡都是 boolean 轉成 0 與 1 
print(np.sum(test_y))

151
164


In [34]:
predictions.__len__()

8217

In [35]:
test_y.__len__()

8217

In [36]:
# 查看 AI 預測的結果，與人工 tag 比較。
categories = [0,0,0,0]
for i in range(predictions.__len__()): 
    if predictions[i]==test_y[i] and test_y[i]==0: categories[0]+=1; continue
    if predictions[i]!=test_y[i] and test_y[i]==1: categories[1]+=1; continue
    if predictions[i]!=test_y[i] and test_y[i]==0: categories[2]+=1; continue
    if predictions[i]==test_y[i] and test_y[i]==1: categories[3]+=1; continue
%f __main__ :> categories ( 沒事的，漏掉的，多的，一致的 ) tib.
        

__main__ :> categories ( 沒事的，漏掉的，多的，一致的 ) tib. \ ==> [8020, 46, 33, 118] (<class 'list'>)


# 輸出 預測結果 到 predictions.csv 檔
可以用 excel 打開，手動合併到總表裡去，就得到了每件物料的 Tag.

In [37]:
# 輸出 預測結果 到 predictions.csv 檔
bc = list(test_data['Barcode'])  # Barcode is the id of each item

if bc.__len__() != predictions.__len__() :
    raise Exception("預測出來的結果總數 %i 與給定的資料筆數 %i 不同！" % (predictions.__len__(),bc.__len__()))

predcsv = "Barcode,Tag\n"
for i in range(predictions.__len__()): 
    predcsv += bc[i] + ',' + str(predictions[i]) + '\n';
    
peforth.push(predcsv).dictate('char predictions.csv writeTextFile');  # save the predictions to csv file so you can merge it to excel 

Automatic pdb calling has been turned ON


### 用 TensorBoard 查看訓練記錄圖表

In [38]:
mask = test_data['Days'].isin([1063,1113])

# mask = [df['Days'] >= 300]
mask

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10       True
11       True
12       True
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
8479    False
8480    False
8481    False
8482    False
8483    False
8484    False
8485    False
8486    False
8487    False
8488    False
8489    False
8490    False
8491    False
8492    False
8493    False
8494    False
8495    False
8496    False
8497    False
8498    False
8499    False
8500    False
8501    False
8502    False
8503    False
8504    False
8505    False
8506    False
8507    False
8508    False
Name: Days, Length: 8217, dtype: bool

In [40]:
# 只關心 over 300d 的方法，很簡單，用 df.query 篩選 dataframe 例如 test_data.query('300 <= Days') 
# See https://stackoverflow.com/questions/12065885/filter-dataframe-rows-if-value-in-column-is-in-a-set-list-of-values
test_data.query('300 <= Days')

,Customer,ProjectName,PartNo,Barcode,BorrowerDEPT,QTY,Days
0,Molly,QP200,81.18R55.108G,2014050900423,20QK10,1,1272
1,Molly,QP200,81.18R55.108G,2014050900430,20QK10,1,1272
2,Molly,QP200,81.18R55.108G,2014050900447,20QK10,1,1272
3,Molly,QP200,81.18R55.108G,2014050900454,20QK10,1,1272
4,Molly,QP200,81.18R55.108G,2014050900461,20QK10,1,1272
5,Molly,QP200,81.18R55.108G,2014050900478,20QK10,1,1272
6,Molly,QP200,81.18R55.108G,2014050900485,20QK10,1,1272
7,Molly,QP200,81.18R55.108G,2014050900492,20QK10,1,1272
8,Molly,QP200,81.18R55.108G,2014050900508,20QK10,1,1272
9,Molly,QP200,81.18R55.108G,2014050900515,20QK10,1,1272


In [ ]:
a[1,2,3]